# Serie 4

## Esercizio 1

### Lock Espliciti
Trasformando la classe `ExplicitLocks` ed aggiungendo un campo statico `lock` possiamo sfruttare le funzionalità di *Lock espliciti*. Nel caso specifico, aggiungendo un explicit lock al metodo `increment()` ed al metodo `getValue()` possiamo assicurarci che le operazioni di lettura / scrittura sono *thread safe*.

In [3]:
import java.util.concurrent.locks.Lock;
import java.util.concurrent.locks.ReentrantLock;

public class ExplicitLocks {
	private static long value = 0;
	public static Lock lock = new ReentrantLock();
    public static long increment() {
		ExplicitLocks.lock.lock();
		value++;
		try {
			Thread.sleep(10);
		} catch (final InterruptedException e) {}
		long rval = value;
		ExplicitLocks.lock.unlock();
		return rval;
	}

	public static long getValue() {
		ExplicitLocks.lock.lock();
		try {
			Thread.sleep(1);
		} catch (final InterruptedException e) { }
		long rval = value;
		ExplicitLocks.lock.unlock();
		return rval;
	}
    // ...
}

### Read-Write locks
Con l'utilizzo dei `ReadWrite` locks possiamo rendere l'accesso a `value` ancora più efficiente, mantenendo sempre la thread safety di cui necessita.

In [4]:
import java.util.concurrent.locks.ReadWriteLock;
import java.util.concurrent.locks.ReentrantReadWriteLock;

public class RWLocks {
	private static long value = 0;
	public static ReadWriteLock lock = new ReentrantReadWriteLock();

	public static long increment() {
		lock.writeLock().lock();
		value++;
		lock.writeLock().unlock();
		try {
			Thread.sleep(10);
		} catch (final InterruptedException e) { }
		return value;
	}

	public static long getValue() {
		lock.readLock().lock();
		try {
			Thread.sleep(1);
		} catch (final InterruptedException e) { }
		long rval = value;
		lock.readLock().unlock();
		return rval;
	}
    // ...
}

## Esercizio 2

Non ho riscontrato particolari problemi nella risoluzione di questo esercizio.

## Esercizio 3

Il problema dell'implementazione sta nel fatto che i cambiamenti non sono sincronizzati in quanto la variabile `version` non è sempre uguale per tutti i thread.  
Una possibile soluzione è quella di rendere la variabile `volatile` ed aggiornare i numeri di telefono in un blocco synchronized per essere sicuri di non avere "dati a metà" nel caso in cui l'uomo di affari cambia il numero di telefono mentre lo stiamo aggiornado.  
Alternativamente è possibile utilizzare dei lock espliciti sui singoli numeri di telefono, ma questo impatterebbe sulle performances. Un sistema di versioning come quello già parzialmente implementato mi pare il modo più efficiente per risolvere il problema.  

In [5]:
class Contact implements Runnable {
	private final int id;

	public Contact(int id) {
		this.id = id;
	}

	@Override
	public void run() {
		System.out.println("Contact" + id + ": started");
		
		int version = -1;
		while  (true) {
			// Wait for version updates

			if (version == S4Esercizio3.version)
				continue;
			// Update local version
			synchronized (this) {
				version = S4Esercizio3.version;
				// Used to terminate
				if (version == -1)
					break;

				// update local numbers
				int home = S4Esercizio3.home;
				int office = S4Esercizio3.office;
				int mobile = S4Esercizio3.mobile;
				int emergency = S4Esercizio3.emergency;
				System.out.println("Contact" + id + ": new Phonenumbers [home=" + home + ", office=" + office + ", mobile=" + mobile + ", emergency=" + emergency + "]");
			}
		}
		System.out.println("Contact" + id + ": terminating");
	}
}